In [1]:
!pip install pymystem3

In [2]:
# !pip install pymorphy2
# !pip install pymorphy2-dicts-ru
# !pip install DAWG-Python

In [3]:
import numpy as np
import os
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

from pymystem3 import Mystem
# from pymorphy2 import MorphAnalyzer
from nltk.tokenize import WordPunctTokenizer
from nltk.corpus import stopwords
from string import punctuation
from tqdm import tqdm

In [4]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/preprocessed-data/df_preprocessed.csv
/kaggle/input/texts-classification-iad-hse-intro-2020/train.csv
/kaggle/input/texts-classification-iad-hse-intro-2020/test.csv
/kaggle/input/texts-classification-iad-hse-intro-2020/sample_submission.csv


In [5]:
DATA_DIR = '/kaggle/input/'

df_train = pd.read_csv(os.path.join(DATA_DIR, 'preprocessed-data/df_preprocessed.csv'), dtype={'Category' : 'uint8'})
# df_test = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'), dtype={'itemid' : 'uint32'})
# sub = pd.read_csv(os.path.join(DATA_DIR, 'sample_submission.csv'), dtype={'itemid' : 'uint32', 'Category' : 'uint8'})

In [6]:
df_train.head()

,Category_name,Category,title&description
0,Запчасти и аксессуары,10,эбу renault nissan dci delphi год комплект эбу...
1,Бытовая техника,21,утюг утп ватт ссср год продавать фото утюг утп...
2,Предложение услуг,114,возвма машина выкуп взять машина аренда послед...
3,"Одежда, обувь, аксессуары",27,полусапожки полусапожки отличный состояние оде...
4,"Одежда, обувь, аксессуары",27,босоножка кожаный кожаный натура босоножка kar...


In [7]:
df_train['title&description'][7]

'свитер продавать свитер идеально состояние фирма benetton размер\n'

In [8]:
df_train['title&description'][4184249]

nan

In [9]:
rows = df_train.index[df_train.isnull().any(axis=1)]
len(rows)

216

In [10]:
df_train.drop(rows, 0, inplace=True)

In [11]:
df_train.shape

(4233826, 3)

In [12]:
len(df_train['Category'].value_counts())

50

In [13]:
df_train.isnull().sum()

Category_name        0
Category             0
title&description    0
dtype: int64

In [14]:
%%time

tfidf_vectorizer = TfidfVectorizer(ngram_range=(1, 2), max_features=25000)
X_train_tfidf = tfidf_vectorizer.fit_transform(df_train['title&description'])

CPU times: user 11min 6s, sys: 17.1 s, total: 11min 23s
Wall time: 11min 23s


In [15]:
len(tfidf_vectorizer.get_feature_names())

25000

In [16]:
y_train = df_train['Category']

In [17]:
del df_train

In [18]:
%%time

lr = LogisticRegression(random_state=13)
lr.fit(X_train_tfidf, y_train)

CPU times: user 45min 7s, sys: 6min 50s, total: 51min 58s
Wall time: 49min 7s


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='auto', n_jobs=None, penalty='l2',
                   random_state=13, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [19]:
df_test = pd.read_csv(os.path.join(DATA_DIR, 'texts-classification-iad-hse-intro-2020/test.csv'), dtype={'itemid' : 'uint32'})

In [20]:
df_test

,title,description,itemid
0,Мастерка,Мастерка фирмы форвард. Белого цвета. В идеаль...,1778449823
1,Зимние сапоги,"Продаю зимние сапоги, в хорошем состоянии, все...",1677656962
2,Видеонаблюдение 8 камер,В комплект Atis AMD-2MIR-8kit входит: /\n1. Ку...,1758182804
3,Запчасти для GLE,Запчасти GLE,1689811299
4,Бластер nerf,Состояние 5+/\nПродаю потому что не нужен/\n18...,1804706240
...,...,...,...
1411343,Бетоносмеситель Россия бсэ-140 чугун,Бетономешалка российского производства - не Ки...,902146704
1411344,Разношрокие колеса R19 оригинал Porsche Panamera,Диски с резиной в отличном состоянии/\n /\nШин...,1799983289
1411345,Комбенз зимний размер,"Комбензон на овчине .Подойдет зима , весна, ос...",1177237748
1411346,Смартфон Honor 5C,В отличном состоянии. Использовался с защитным...,1793055786


In [21]:
df_test.isnull().sum()

title          0
description    1
itemid         0
dtype: int64

In [22]:
df_test.fillna('', inplace=True)

In [23]:
df_test.isnull().sum()

title          0
description    0
itemid         0
dtype: int64

In [24]:
%%time

df_test['title&description'] = df_test['title'].str[:] + ' ' + df_test['description'].str[:]
df_test.head()

CPU times: user 3.54 s, sys: 409 ms, total: 3.95 s
Wall time: 3.96 s


,title,description,itemid,title&description
0,Мастерка,Мастерка фирмы форвард. Белого цвета. В идеаль...,1778449823,Мастерка Мастерка фирмы форвард. Белого цвета....
1,Зимние сапоги,"Продаю зимние сапоги, в хорошем состоянии, все...",1677656962,"Зимние сапоги Продаю зимние сапоги, в хорошем ..."
2,Видеонаблюдение 8 камер,В комплект Atis AMD-2MIR-8kit входит: /\n1. Ку...,1758182804,Видеонаблюдение 8 камер В комплект Atis AMD-2M...
3,Запчасти для GLE,Запчасти GLE,1689811299,Запчасти для GLE Запчасти GLE
4,Бластер nerf,Состояние 5+/\nПродаю потому что не нужен/\n18...,1804706240,Бластер nerf Состояние 5+/\nПродаю потому что ...


In [25]:
df_test = df_test.drop(['title','description'], axis=1)

In [26]:
df_test

,itemid,title&description
0,1778449823,Мастерка Мастерка фирмы форвард. Белого цвета....
1,1677656962,"Зимние сапоги Продаю зимние сапоги, в хорошем ..."
2,1758182804,Видеонаблюдение 8 камер В комплект Atis AMD-2M...
3,1689811299,Запчасти для GLE Запчасти GLE
4,1804706240,Бластер nerf Состояние 5+/\nПродаю потому что ...
...,...,...
1411343,902146704,Бетоносмеситель Россия бсэ-140 чугун Бетономеш...
1411344,1799983289,Разношрокие колеса R19 оригинал Porsche Paname...
1411345,1177237748,Комбенз зимний размер Комбензон на овчине .Под...
1411346,1793055786,Смартфон Honor 5C В отличном состоянии. Исполь...


In [27]:
%%time

df_test['title&description'] = df_test['title&description'].str.lower()
df_test['title&description'] = df_test['title&description'].str.replace(r"http\S+", "")
df_test['title&description'] = df_test['title&description'].str.replace(r"@\S+", "")

CPU times: user 8.78 s, sys: 557 ms, total: 9.33 s
Wall time: 9.33 s


In [28]:
noise = stopwords.words('russian') + list(punctuation)
tknzr = WordPunctTokenizer()
# morph = MorphAnalyzer()
mystem = Mystem()

Installing mystem to /root/.local/bin/mystem from http://download.cdn.yandex.net/mystem/mystem-3.1-linux-64bit.tar.gz


In [29]:
def data_preprocessing(text):
    tokens = tknzr.tokenize(text)
    text = [word for word in tokens if (word not in noise and word.isalpha())]
    text = ' '.join(text)
    return text

In [30]:
def lemmatization(text):
    return ''.join(mystem.lemmatize(text))

In [31]:
for idx, text in tqdm(enumerate(df_test['title&description'])):
    df_test.at[idx, 'title&description'] = data_preprocessing(text)

1411348it [06:21, 3695.07it/s]


In [32]:
for idx, text in tqdm(enumerate(df_test['title&description'])):
    df_test.at[idx, 'title&description'] = lemmatization(text)

500072it [12:14, 703.17it/s]

In [33]:
%%time

X_test_tfidf = tfidf_vectorizer.transform(df_test['title&description'])
y_test_pred = lr.predict(X_test_tfidf)

CPU times: user 2min 51s, sys: 3.26 s, total: 2min 54s
Wall time: 2min 54s


In [34]:
sub = pd.read_csv(os.path.join(DATA_DIR, 'texts-classification-iad-hse-intro-2020/sample_submission.csv'), dtype={'itemid' : 'uint32', 'Category' : 'uint8'})

In [35]:
sub.shape, y_test_pred.shape

((1411348, 2), (1411348,))

In [36]:
sub['Category'] = y_test_pred
sub.head()

,Id,Category
0,1778449823,27
1,1677656962,29
2,1758182804,32
3,1689811299,10
4,1804706240,36


In [37]:
len(sub['Category'].value_counts())

49

In [38]:
sub.to_csv('submission.csv', index=False)